# Fundamentals of Information Systems

## Python Programming (for Data Science)

### Master's Degree in Data Science

#### Giorgio Maria Di Nunzio
#### (Courtesy of Gabriele Tolomei FIS 2018-2019)
<a href="mailto:giorgiomaria.dinunzio@unipd.it">giorgiomaria.dinunzio@unipd.it</a><br/>
University of Padua, Italy<br/>
2021/2022<br/>

# Lecture 9: Data Preparation with <code>pandas</code>

## What Does Data Preparation Mean?

-  It typically consists of **loading**, **cleaning**, **transforming**, and **rearranging** data (the last three steps are also referred to as data **"munging"**). 

-  Such tasks are often reported to take up 80% or more of your development time (for a machine learning/data science task). 

-  Sometimes it can be achieved by using a mixture of tools, i.e., from general-purpose programming languages, like Python, Perl, R, or Java, to UNIX tools like <code>**sed**</code> or <code>**awk**</code>.

-  Luckily, <code>**pandas**</code> provides you with a single, high-level, flexible, and fast set of tools to enable you to manipulate data into the right form.

## Our Focus

1.  Handling missing data (**NA** or **N**ot **A**vailable)

2.  Dealing with duplicates

3.  Managing very extreme values (i.e., **outliers**)

4.  Combining multiple datasets into a single one

In [1]:
"""
Before we start our journey on data preparation with pandas,
we get back to the example we used in Lecture 08.
"""
import numpy as np
import pandas as pd

url = 'https://raw.githubusercontent.com/justmarkham/DAT8/master/data/u.user'
# The first line of the file represents the header, and each field
# is separated by a pipe
"""
We specify the url where the data is located, the character u|sed to separate fields ('|')
and the name of the column to use as row label (otherwise, RangeInteger will be used)
"""
users = pd.read_csv(url, sep='|', index_col='user_id')
print(users.head())

         age gender  occupation zip_code
user_id                                 
1         24      M  technician    85711
2         53      F       other    94043
3         23      M      writer    32067
4         24      M  technician    43537
5         33      F       other    15213


## Handling Missing Data (*NA*)

-  Missing data (**NA**) may either be data that does not exist or that exists but was not observed (e.g., due to measurement issues).

-  <code>**pandas**</code> makes working with missing data as painless as possible. For example, all of the descriptive statistics on <code>**pandas**</code> objects exclude **NA** by default.

-  <code>**pandas**</code> represents **NA** using so-called **sentinel values**.

-  Two of the most common sentinel values are <code>**None**</code> and the floating point value <code>**NaN**</code> (**N**ot **a** **N**umber).

In [2]:
"""
Let's see if there are any missing data (NA) on our loaded dataset.
"""

# isnull() returns a boolean DataFrame with the same shape of the DataFrame object
# where you invoke the method. Each entry of this new boolean DataFrame either contains
# True or False depending on whether the corresponding entry in the original DataFrame
# is missing or not
print("Shape of the boolean DataFrame: {}\n".format(users.isnull().shape))
print(users.isnull().head())

Shape of the boolean DataFrame: (943, 4)

           age  gender  occupation  zip_code
user_id                                     
1        False   False       False     False
2        False   False       False     False
3        False   False       False     False
4        False   False       False     False
5        False   False       False     False


In [3]:
"""
In order to see which column has at least one missing value,
we can call the method any() on the boolean DataFrame above.
This returns a boolean Series which contains an entry for each column 
(row aggregation) which evaluates to True if at least one element
of that column is True, False otherwise.
"""
print("Shape of the boolean Series: {}\n".format(users.isnull().any().shape))
print(users.isnull().any().head())

Shape of the boolean Series: (4,)

age           False
gender        False
occupation    False
zip_code      False
dtype: bool


In [4]:
"""
We have verified that our dataset does not contain any missing value.
What if, though, I would like to obtain a row-wise (i.e., column aggregation)
boolean Series containing a value for each row, which tells me whether that
row contains or not at least one NA value?
"""

print("Shape of the boolean Series: {}\n".format(users.isnull().any(axis=1).shape))
print(users.isnull().any(axis=1).head())

Shape of the boolean Series: (943,)

user_id
1    False
2    False
3    False
4    False
5    False
dtype: bool


In [5]:
"""
In order to find whether there exists at least one NA on the whole DataFrame
we can simply aggregate one more time the boolean Series above using another any()
"""

print("Q: Is there at least a missing value in our DataFrame? A: {}".
      format(users.isnull().any().any()))

Q: Is there at least a missing value in our DataFrame? A: False


In [6]:
"""
Of course, the same thing can be achieved using different approaches.
This is possibly the quickest solution.
"""

# values returns a 2-D numpy array (ndarray) and any() "flatten" it 
print("Q: Is there at least a missing value in our DataFrame? A: {}".
      format(users.isnull().values.any()))

Q: Is there at least a missing value in our DataFrame? A: False


In [7]:
"""
This solution makes exactly the same 3 steps as above but instead of computing
boolean aggregation from the boolean DataFrame, it sum boolean values over the rows
(i.e., column-wise) and finally makes a final aggregation step (sum) to obtain
the number of missing values of the DataFrame.
NOTE: this is generally slower but it provides you with an extra information
(i.e., how many NA values are in the DataFrame, not just a boolean value!)
"""
print("There are {} missing values (NA) in the DataFrame".
      format(users.isnull().sum().sum()))

There are 0 missing values (NA) in the DataFrame


In [8]:
"""
Suppose we want to randomly perturbate our dataset with some missing values.
First of all, let's create a deep copy of our original DataFrame.
"""

# Prepare the deep copy of the DataFrame where we are going to randomly insert
# some missing values (NA)
users_with_na = users.copy()

In [9]:
"""
Let's create a uniform random sample of size=10 drawn from the range [1, 943]
"""
np.random.seed(42)
row_indices = np.random.randint(low = 1, high = 944, size = 10)
# alternatively, use np.random.choice(np.arange(1, 944), size=10, replacement=False)
print("Random row indices: {}".format(row_indices))

Random row indices: [103 436 861 271 107  72 701  21 615 122]


In [10]:
"""
We are going to use the first half of the randomly selected indices
for populating with np.NaN the column 'age', whilst the second half
is going to be used to set to None the column 'occupation'.
The two middle indices will be used for both. 
In other words:
- rows labeled as 103, 436, 861, 271 will be used to
set column 'age' to np.NaN
- rows labeled as 701, 21, 615, 122 will be used to
set column 'occupation' as None
- rows labeled as 107 and 72 will be used to set both
column 'age' to np.NaN and column 'occupation' to None
"""

"\nWe are going to use the first half of the randomly selected indices\nfor populating with np.NaN the column 'age', whilst the second half\nis going to be used to set to None the column 'occupation'.\nThe two middle indices will be used for both. \nIn other words:\n- rows labeled as 103, 436, 861, 271 will be used to\nset column 'age' to np.NaN\n- rows labeled as 701, 21, 615, 122 will be used to\nset column 'occupation' as None\n- rows labeled as 107 and 72 will be used to set both\ncolumn 'age' to np.NaN and column 'occupation' to None\n"

In [11]:
print("Let's first extract the records we want to update for column 'age':\n{}".
      format(users_with_na.loc[row_indices[:6], 'age']))

Let's first extract the records we want to update for column 'age':
user_id
103    26
436    30
861    38
271    51
107    39
72     48
Name: age, dtype: int64


In [13]:
print("Let's now extract the records we want to update for column 'occupation':\n{}".
      format(users_with_na.loc[row_indices[4:], 'occupation']))
# I swear, I didn't do it on purpose! 
# This was truly the outcome of a purely (pseudo-)random experiment but apparently
# 'scientist' will be sacrificed and set to None

Let's now extract the records we want to update for column 'occupation':
user_id
107        scientist
72     administrator
701        librarian
21            writer
615         educator
122           writer
Name: occupation, dtype: object


In [14]:
"""
Let's set the values as we planned.
"""
users_with_na.loc[row_indices[:6], 'age'] = np.nan
users_with_na.loc[row_indices[4:], 'occupation'] = None
# NOTE: the same won't work if we use something like the following:
# users_with_na.loc[row_indices[:6]]['age'] = np.nan
# users_with_na.loc[row_indices[4:]]['occupation'] = None
# This is because in case of [] operator, we are actually 
# accessing a copy of the selected slice, whereas above we are working on a view.
# More information on this - a.k.a. SettingWithCopyWarning - is available here: 
# https://www.dataquest.io/blog/settingwithcopywarning/

In [15]:
"""
Let's verify if changes actually took place!
"""
print("Let's see how column 'age' looks like:\n{}".
      format(users_with_na.loc[row_indices[:7], 'age']))
print()
print("Let's see how column 'occupation' looks like:\n{}".
      format(users_with_na.loc[row_indices[3:], 'occupation']))

Let's see how column 'age' looks like:
user_id
103     NaN
436     NaN
861     NaN
271     NaN
107     NaN
72      NaN
701    51.0
Name: age, dtype: float64

Let's see how column 'occupation' looks like:
user_id
271    engineer
107        None
72         None
701        None
21         None
615        None
122        None
Name: occupation, dtype: object


In [16]:
"""
Now, let's try to see if the tests above we run for finding any NA work correctly.
"""
print("Q: Is there at least a missing value in our DataFrame? A: {}".
      format(users_with_na.isnull().any().any()))
print()
print("There are {} missing values (NA) in the DataFrame".
      format(users_with_na.isnull().sum().sum()))

Q: Is there at least a missing value in our DataFrame? A: True

There are 12 missing values (NA) in the DataFrame


## Filtering Missing Data

In [17]:
"""
You may want to drop rows or columns of a DataFrame which are all NA 
or only those containing any NAs. 
The pandas.dropna function by default drops any row containing a missing value.
By default, dropna returns a new object but we can specify inplace=True for
any in-place change.
"""

# This will drop all the rows containing at least one NA
cleaned_users = users_with_na.dropna()

# After issuing the above command, we are expecting the DataFrame to have 10 rows less
# (i.e., 933 instead of 943)
print("Number of records in the original DataFrame = {}\n".format(users_with_na.shape[0]))
print("Number of records in the cleaned DataFrame = {}".format(cleaned_users.shape[0]))

Number of records in the original DataFrame = 943

Number of records in the cleaned DataFrame = 933


In [18]:
"""
If we instead want to delete only those rows which are FULL of NAs,
then we have to pass how='all' to dropna.
"""
# This will drop all the rows containing ALL NAs
cleaned_users_all = users_with_na.dropna(how = 'all')

# After issuing the above command, how many rows will be dropped?
print("Number of records in the original DataFrame = {}\n".format(users_with_na.shape[0]))
print("Number of records in the cleaned DataFrame = {}".format(cleaned_users_all.shape[0]))

Number of records in the original DataFrame = 943

Number of records in the cleaned DataFrame = 943


In [19]:
"""
As usual, dropna works on rows (axis 0) by default.
Instead, if we want to delete columns corresponding to a missing value
we can pass axis=1 argument to dropna.
"""
# This will drop all the columns containing at least one NA
cleaned_users_columns = users_with_na.dropna(axis = 1)

In [20]:
# How many columns are we expecting the above command will drop?

In [21]:
# After issuing the above command, weare expecting the DataFrame to have 2 columns less
# (i.e., 2 instead of 4, as 'age' and 'occupation' have both at least one NA value)
print("Number of columns in the original DataFrame = {}\n".format(users_with_na.shape[1]))
print("Number of columns in the cleaned DataFrame = {}".format(cleaned_users_columns.shape[1]))

Number of columns in the original DataFrame = 4

Number of columns in the cleaned DataFrame = 2


## Filling in Missing Data

In [22]:
"""
'fillna' fill in missing data with some values, rather than filtering them out.
"""
# Suppose we fill all missing values with 0 (not in-place, otherwise set inplace=True)
users_fill_na = users_with_na.fillna(0)

# Let's verify if changes actually took place!
print("Let's see how column 'age' looks like:\n{}".
      format(users_fill_na.loc[row_indices[:6], 'age']))
print()
print("Let's see how column 'occupation' looks like:\n{}".
      format(users_fill_na.loc[row_indices[4:], 'occupation']))

Let's see how column 'age' looks like:
user_id
103    0.0
436    0.0
861    0.0
271    0.0
107    0.0
72     0.0
Name: age, dtype: float64

Let's see how column 'occupation' looks like:
user_id
107    0
72     0
701    0
21     0
615    0
122    0
Name: occupation, dtype: object


In [23]:
"""
We can also pass a dictionary to fillna in order to specify
different values to replace NA with.
"""
users_fill_na = users_with_na.fillna({'age': 0, 'occupation': 'none'})
# Let's verify if changes actually took place!
print("Let's see how column 'age' looks like:\n{}".
      format(users_fill_na.loc[row_indices[:6], 'age']))
print()
print("Let's see how column 'occupation' looks like:\n{}".
      format(users_fill_na.loc[row_indices[4:], 'occupation']))

Let's see how column 'age' looks like:
user_id
103    0.0
436    0.0
861    0.0
271    0.0
107    0.0
72     0.0
Name: age, dtype: float64

Let's see how column 'occupation' looks like:
user_id
107    none
72     none
701    none
21     none
615    none
122    none
Name: occupation, dtype: object


## Removing Duplicates

In [24]:
"""
The DataFrame method duplicated() returns a boolean Series 
indicating whether each row is a duplicate or not.
"""
# Let's go back to our original DataFrame
print("Duplicated rows:\n{}".format(users.duplicated().head()))
print()

Duplicated rows:
user_id
1    False
2    False
3    False
4    False
5    False
dtype: bool



In [25]:
# What if I would like to see if there exists at least one duplicated row?
print("Q: Is there at least one duplicated row? A: {}"
      .format(users.duplicated().any()))

Q: Is there at least one duplicated row? A: True


In [26]:
# Suppose I want to extract only the duplicated rows
print("Duplicated rows:\n{}".format(users))

Duplicated rows:
         age gender     occupation zip_code
user_id                                    
1         24      M     technician    85711
2         53      F          other    94043
3         23      M         writer    32067
4         24      M     technician    43537
5         33      F          other    15213
...      ...    ...            ...      ...
939       26      F        student    33319
940       32      M  administrator    02215
941       20      M        student    97229
942       48      F      librarian    78209
943       22      M        student    77841

[943 rows x 4 columns]


In [27]:
users.duplicated()

user_id
1      False
2      False
3      False
4      False
5      False
       ...  
939    False
940    False
941    False
942    False
943    False
Length: 943, dtype: bool

In [28]:
# Suppose I want to extract only the duplicated rows
print("Duplicated rows:\n{}".format(users[users.duplicated()]))

Duplicated rows:
         age gender occupation zip_code
user_id                                
496       21      F    student    55414
572       51      M   educator    20003
621       17      M    student    60402
684       28      M    student    55414
733       44      F      other    60630
805       27      F      other    20009
890       32      M    student    97301


In [29]:
"""
By default, if you have 3 duplicated rows, only the last two will be
marked as duplicates (i.e., the first occurrence is kept).
This can be changed by specifying the parameters 'keep'
keep : {'first', 'last', False}, default 'first'
first : Mark duplicates as True except for the first occurrence.
last : Mark duplicates as True except for the last occurrence.
False : Mark all duplicates as True.
"""
# Suppose I want to extract only the duplicated rows, this time considering them all
print("Duplicated rows:\n{}".format(users[users.duplicated(keep = False)]))

Duplicated rows:
         age gender occupation zip_code
user_id                                
67        17      M    student    60402
85        51      M   educator    20003
198       21      F    student    55414
350       32      M    student    97301
428       28      M    student    55414
437       27      F      other    20009
460       44      F      other    60630
496       21      F    student    55414
572       51      M   educator    20003
621       17      M    student    60402
684       28      M    student    55414
733       44      F      other    60630
805       27      F      other    20009
890       32      M    student    97301


In [30]:
"""
Relatedly, drop_duplicates() returns a DataFrame where the duplicated array is False.
"""
# Remove duplicated rows (keeping the first occurrence of each duplicates)
users_with_no_dup = users.drop_duplicates()
# What if I would like to see if there exists at least one duplicate row, now?
print("Q: Is there at least one duplicated row? A: {}"
      .format(users_with_no_dup.duplicated().any()))
print()
print("Total number of rows after removing duplicated rows = {}"
      .format(users_with_no_dup.shape[0]))

Q: Is there at least one duplicated row? A: False

Total number of rows after removing duplicated rows = 936


In [31]:
"""
We can also specify the same 'keep' argument to decide on how to
mark duplicates, and therefore remove them
"""
# Remove ALL duplicated rows
users_with_no_dup = users.drop_duplicates(keep = False)
# What if I would like to see if there exists at least one duplicate row, now?
print("Q: Is there at least one duplicated row? A: {}"
      .format(users_with_no_dup.duplicated().any()))
print()
print("Total number of rows after removing duplicated rows = {}"
      .format(users_with_no_dup.shape[0]))

Q: Is there at least one duplicated row? A: False

Total number of rows after removing duplicated rows = 929


In [32]:
"""
By default, both duplicated() and drop_duplicates() consider all of the columns; 
alternatively we can specify any subset of them to detect duplicates. 
Suppose we want to filter duplicates only based on the 'gender' and 'occupation' columns.
"""
# Suppose I want to extract only the duplicated rows w.r.t. 'gender' and 'occupation'
print("There are {} duplicated rows having the same 'gender' and 'occupation'.\n\
The following are the first 5 of them:\n{}"
      .format(users[users.duplicated(['gender', 'occupation'])].shape[0],
              users[users.duplicated(['gender', 'occupation'])].head()))

There are 902 duplicated rows having the same 'gender' and 'occupation'.
The following are the first 5 of them:
         age gender     occupation zip_code
user_id                                    
4         24      M     technician    43537
5         33      F          other    15213
8         36      M  administrator    05201
11        39      F          other    30329
12        28      F          other    06405


## Transforming Data Using a Function or Mapping

In [33]:
"""
For many data sets, we may wish to perform some transformation based on the values
in an array, Series, or column in a DataFrame.
"""
# Suppose we want to add a column indicating the salary for each occupation. 
# Let's write down a mapping of each occupation to salary
occupation_to_salary = {'technician': 25000, 'administrator': 150000,
                        'writer': 40000, 'executive': 300000, 'other': 18000,
                        'student': 1300, 'lawyer': 27500, 'educator': 45000,
                        'scientist': 60000, 'entertainment': 185000, 'programmer': 55000,
                        'librarian': 22000, 'homemaker': 240000, 'artist': 72000,
                        'engineer': 91000, 'marketing': 66000, 'none': 0,
                        'healthcare': 41000, 'retired': 52000, 'salesman': 48000,
                        'doctor': 140000
                       }

In [34]:
occupation_to_salary

{'technician': 25000,
 'administrator': 150000,
 'writer': 40000,
 'executive': 300000,
 'other': 18000,
 'student': 1300,
 'lawyer': 27500,
 'educator': 45000,
 'scientist': 60000,
 'entertainment': 185000,
 'programmer': 55000,
 'librarian': 22000,
 'homemaker': 240000,
 'artist': 72000,
 'engineer': 91000,
 'marketing': 66000,
 'none': 0,
 'healthcare': 41000,
 'retired': 52000,
 'salesman': 48000,
 'doctor': 140000}

In [35]:
users['occupation'].map(occupation_to_salary)

user_id
1       25000
2       18000
3       40000
4       25000
5       18000
        ...  
939      1300
940    150000
941      1300
942     22000
943      1300
Name: occupation, Length: 943, dtype: int64

In [36]:
"""
The map method on a Series accepts a function or dict-like object containing a mapping.
"""
users['salary'] = users['occupation'].map(occupation_to_salary)
print(users.head())

         age gender  occupation zip_code  salary
user_id                                         
1         24      M  technician    85711   25000
2         53      F       other    94043   18000
3         23      M      writer    32067   40000
4         24      M  technician    43537   25000
5         33      F       other    15213   18000


In [37]:
"""
Alternatively, we could also have passed to map a lambda function that does all the work.
"""
# Let's first delete the salary column
del users['salary']
# Verify the column is really removed
print(users.head())

         age gender  occupation zip_code
user_id                                 
1         24      M  technician    85711
2         53      F       other    94043
3         23      M      writer    32067
4         24      M  technician    43537
5         33      F       other    15213


In [38]:
# lambda function
users['salary'] = users['occupation'].map(lambda x: occupation_to_salary[x])
print(users.head())

         age gender  occupation zip_code  salary
user_id                                         
1         24      M  technician    85711   25000
2         53      F       other    94043   18000
3         23      M      writer    32067   40000
4         24      M  technician    43537   25000
5         33      F       other    15213   18000


In [39]:
# lambda function
users['salary'] = users['occupation'].map(lambda o: occupation_to_salary[o])
print(users.head())

         age gender  occupation zip_code  salary
user_id                                         
1         24      M  technician    85711   25000
2         53      F       other    94043   18000
3         23      M      writer    32067   40000
4         24      M  technician    43537   25000
5         33      F       other    15213   18000


## Replacing Values

In [40]:
"""
Filling in missing data with the 'fillna' method is a special case 
of more general value replacement. 
While 'map' can be used to modify a subset of values in an object, 
'replace' provides a simpler and more flexible way to do so.
"""
# Suppose we would like to consider 'none' value of 'occupation' as missing data
# (i.e., 'none' can be considered as a sentinel value)
# NOTE: remember that this can be also specified when loading the DataFrame
# by specifying na_values
users['occupation'] = users['occupation'].replace('none', np.nan)
print("The number of replaced rows is = {}"
      .format(users[(users['occupation'].isnull())].shape[0]))
print("The following are the first 5 rows that have been replaced:\n{}"
      .format(users[(users['occupation'].isnull())].head()))

The number of replaced rows is = 9
The following are the first 5 rows that have been replaced:
         age gender occupation zip_code  salary
user_id                                        
57        16      M        NaN    84010       0
127       33      M        NaN    73439       0
130       20      M        NaN    60115       0
256       35      F        NaN    39042       0
289       11      M        NaN    94619       0


In [41]:
# Let's verify that 'occupation' contains actually some NaN
print("Q: Is there any missing value for 'occupation'? A: {}".
      format(users['occupation'].isnull().any()))

Q: Is there any missing value for 'occupation'? A: True


In [42]:
# Suppose I want to update the salary column corresponding to
# those rows where 'occupation' is now missing
mask = users['occupation'].notnull()
# Update 'salary' where 'occupation' is null. The semantics is as follows.
# 'salary' will keep its value if the mask condition is verified
# (i.e., if 'occupation' is NOT null, otherwise we set it to NaN)
users['salary'] = users['salary'].where(mask, np.nan)
#np.where(mask, users.salary, np.nan)

In [43]:
print("The number of replaced rows is = {}"
      .format(users[(users['occupation'].isnull()) & (users['salary'].isnull())].shape[0]))
print("The following are the first 5 rows that have been replaced:\n{}"
      .format(users[(users['occupation'].isnull()) & (users['salary'].isnull())].head()))

The number of replaced rows is = 9
The following are the first 5 rows that have been replaced:
         age gender occupation zip_code  salary
user_id                                        
57        16      M        NaN    84010     NaN
127       33      M        NaN    73439     NaN
130       20      M        NaN    60115     NaN
256       35      F        NaN    39042     NaN
289       11      M        NaN    94619     NaN


In [44]:
# Let's go back to the original occupation and salary
users = users.fillna({'occupation': 'none'})
users['salary'] = users['occupation'].map(lambda o: occupation_to_salary[o])

## Discretization and Binning

-  Continuous data is often **discretized** or otherwised separated into "**bins**" for analysis. 

-  The typical example is given by the case where you have user's data containing information like 'age', and you what to divide users in a set of fixed age intervals.

In [45]:
"""
Let's consider our 'age' column and suppose we want to divide these into bins, 
such as users aged between 0 and 17, 18 to 25, 26 to 35, 
36 to 55, and finally 56 and older. To do so, we can use pandas.cut function.
"""
# Let's first define a list containing the left-most extreme of each bin
bins = [0, 18, 26, 36, 56, 100]
"""
Consistent with mathematical notation for intervals, a parenthesis ')' 
means that the side is OPEN while the square bracket '[' means it is closed (inclusive). 
By default, intervals are left-open, i.e., (a, b]
Passing 'right=False' those become right-open, i.e., [a, b) 
"""
age_intervals = pd.cut(users['age'], bins, right=False)
print("Ages: {}".format(users['age'].head()))
print("Categorical ranges: {}".format(age_intervals.head()))

Ages: user_id
1    24
2    53
3    23
4    24
5    33
Name: age, dtype: int64
Categorical ranges: user_id
1    [18, 26)
2    [36, 56)
3    [18, 26)
4    [18, 26)
5    [26, 36)
Name: age, dtype: category
Categories (5, interval[int64, left]): [[0, 18) < [18, 26) < [26, 36) < [36, 56) < [56, 100)]


In [46]:
"""
The object pandas returns is a special Categorical object. 
We can treat it like an array of strings indicating the bin name; 
internally it contains a categories array indicating the distinct category names 
along with a labeling for the ages data in the 'codes' attribute
"""
print("Categorical bin codes: {}".format(age_intervals.cat.codes.head()))
print("Categorical bin names: {}".format(age_intervals.cat.categories))

Categorical bin codes: user_id
1    1
2    3
3    1
4    1
5    2
dtype: int8
Categorical bin names: IntervalIndex([[0, 18), [18, 26), [26, 36), [36, 56), [56, 100)], dtype='interval[int64, left]')


In [47]:
"""
Instead of integer labeling for the bin, we can specify which label
to assign to each bin.
"""
age_labels = ['Young', 'Young_Adult', 'Adult', 'Middle_Aged', 'Senior']

age_intervals = pd.cut(users['age'], bins, labels=age_labels, right=False)

print("Ages: {}".format(users['age'].head()))
print("Categorical ranges: {}".format(age_intervals.head()))

Ages: user_id
1    24
2    53
3    23
4    24
5    33
Name: age, dtype: int64
Categorical ranges: user_id
1    Young_Adult
2    Middle_Aged
3    Young_Adult
4    Young_Adult
5          Adult
Name: age, dtype: category
Categories (5, object): ['Young' < 'Young_Adult' < 'Adult' < 'Middle_Aged' < 'Senior']


In [48]:
"""
Let's create an extra column 'age_interval' on the DataFrame with this information.
"""
users['age_interval'] = pd.cut(users['age'], bins, labels=age_labels, right=False)
print(users.head())

         age gender  occupation zip_code  salary age_interval
user_id                                                      
1         24      M  technician    85711   25000  Young_Adult
2         53      F       other    94043   18000  Middle_Aged
3         23      M      writer    32067   40000  Young_Adult
4         24      M  technician    43537   25000  Young_Adult
5         33      F       other    15213   18000        Adult


In [49]:
"""
Sometimes we don't want to specify the intervals ourselves, instead
we want to just specify the number of bins and let pandas figure out how 
data are distributed across those bins.
"""
# If we pass 'cut' an integer number of bins instead of explicit bin edges, 
# it will compute equal-length bins based on the minimum and maximum values in the data. 
age_intervals = pd.cut(users['age'], 5, right=False)
print("Ages: {}".format(users['age'].head()))
print("Categorical ranges: {}".format(age_intervals.head()))

Ages: user_id
1    24
2    53
3    23
4    24
5    33
Name: age, dtype: int64
Categorical ranges: user_id
1    [20.2, 33.4)
2    [46.6, 59.8)
3    [20.2, 33.4)
4    [20.2, 33.4)
5    [20.2, 33.4)
Name: age, dtype: category
Categories (5, interval[float64, left]): [[7.0, 20.2) < [20.2, 33.4) < [33.4, 46.6) < [46.6, 59.8) < [59.8, 73.066)]


In [50]:
# The same as above, but with labels
age_labels = ['Young', 'Young_Adult', 'Adult', 'Middle_Aged', 'Senior']
age_intervals = pd.cut(users['age'], 5, labels=age_labels, right=False)
print("Ages: {}".format(users['age'].head()))
print("Categorical ranges: {}".format(age_intervals.head()))

Ages: user_id
1    24
2    53
3    23
4    24
5    33
Name: age, dtype: int64
Categorical ranges: user_id
1    Young_Adult
2    Middle_Aged
3    Young_Adult
4    Young_Adult
5    Young_Adult
Name: age, dtype: category
Categories (5, object): ['Young' < 'Young_Adult' < 'Adult' < 'Middle_Aged' < 'Senior']


In [51]:
"""
A closely related function 'qcut' bins the data based on sample quantiles. 
Depending on the distribution of the data, using 'cut' will not usually result 
in each bin having the same number of data points. 
Instead, with 'qcut' by definition we will obtain roughly equal-size bins.
"""
# We are using quartiles here [0.25, 0.50, 0.75, 1]
# Alternatively, we can specify the list of our own quantiles 
# (i.e., numbers between 0 and 1, inclusive)
age_intervals = pd.qcut(users['age'], 4)
print("Ages: {}".format(users['age'].head()))
print("Categorical ranges: {}".format(age_intervals.head()))

Ages: user_id
1    24
2    53
3    23
4    24
5    33
Name: age, dtype: int64
Categorical ranges: user_id
1    (6.999, 25.0]
2     (43.0, 73.0]
3    (6.999, 25.0]
4    (6.999, 25.0]
5     (31.0, 43.0]
Name: age, dtype: category
Categories (4, interval[float64, right]): [(6.999, 25.0] < (25.0, 31.0] < (31.0, 43.0] < (43.0, 73.0]]


In [52]:
# The same as above yet with labels
age_labels = ['Young', 'Adult', 'Middle_Aged', 'Senior']
age_intervals = pd.qcut(users['age'], 4, labels=age_labels)
print("Ages: {}".format(users['age'].head()))
print("Categorical ranges: {}".format(age_intervals.head()))

Ages: user_id
1    24
2    53
3    23
4    24
5    33
Name: age, dtype: int64
Categorical ranges: user_id
1          Young
2         Senior
3          Young
4          Young
5    Middle_Aged
Name: age, dtype: category
Categories (4, object): ['Young' < 'Adult' < 'Middle_Aged' < 'Senior']


## Detecting and Filtering Outliers

In [53]:
"""
Let's see the output of the 'describe()' function on our DataFrame.
"""
print(users.describe(include='all'))

               age gender occupation zip_code         salary age_interval
count   943.000000    943        943      943     943.000000          943
unique         NaN      2         21      795            NaN            5
top            NaN      M    student    55414            NaN  Middle_Aged
freq           NaN    670        196        9            NaN          320
mean     34.051962    NaN        NaN      NaN   58394.273595          NaN
std      12.192740    NaN        NaN      NaN   66491.286158          NaN
min       7.000000    NaN        NaN      NaN       0.000000          NaN
25%      25.000000    NaN        NaN      NaN   18000.000000          NaN
50%      31.000000    NaN        NaN      NaN   45000.000000          NaN
75%      43.000000    NaN        NaN      NaN   72000.000000          NaN
max      73.000000    NaN        NaN      NaN  300000.000000          NaN


In [54]:
"""
Suppose we want to see what are the records where 'salary' is greater than 280k.
"""
salary_outlier = users.salary > 280000
print("There are {} salary outliers, 5 of which are as follows:\n{}"
      .format(users[salary_outlier].shape[0], users[salary_outlier].head()))

There are 32 salary outliers, 5 of which are as follows:
         age gender occupation zip_code  salary age_interval
user_id                                                     
6         42      M  executive    98101  300000  Middle_Aged
54        22      M  executive    66315  300000  Young_Adult
84        32      M  executive    55369  300000        Adult
93        48      M  executive    23112  300000  Middle_Aged
98        49      F  executive    90291  300000  Middle_Aged


In [55]:
"""
If we want to change the value of an outlier with a 'cap' value (e.g., 280000)
"""
# 1. Using 'where'
users.salary = users.salary.where(~salary_outlier, 280000)
print(users[salary_outlier].head())

         age gender occupation zip_code  salary age_interval
user_id                                                     
6         42      M  executive    98101  280000  Middle_Aged
54        22      M  executive    66315  280000  Young_Adult
84        32      M  executive    55369  280000        Adult
93        48      M  executive    23112  280000  Middle_Aged
98        49      F  executive    90291  280000  Middle_Aged


In [56]:
"""
Let's revert back to the original salary
"""
users.salary = users.salary.where(~salary_outlier, 300000)
print(users[salary_outlier].head())

         age gender occupation zip_code  salary age_interval
user_id                                                     
6         42      M  executive    98101  300000  Middle_Aged
54        22      M  executive    66315  300000  Young_Adult
84        32      M  executive    55369  300000        Adult
93        48      M  executive    23112  300000  Middle_Aged
98        49      F  executive    90291  300000  Middle_Aged


In [57]:
"""
If we want to change the value of an outlier with a 'cap' value (e.g., 280000)
"""
# 2. Using 'loc'
users.loc[salary_outlier, 'salary'] = 280000
print(users[salary_outlier].head())

         age gender occupation zip_code  salary age_interval
user_id                                                     
6         42      M  executive    98101  280000  Middle_Aged
54        22      M  executive    66315  280000  Young_Adult
84        32      M  executive    55369  280000        Adult
93        48      M  executive    23112  280000  Middle_Aged
98        49      F  executive    90291  280000  Middle_Aged


## Combining <code>pandas</code> Objects

-  Data contained in <code>**pandas**</code> objects (i.e., <code>**Series**</code> and <code>**DataFrame**</code>) can be combined together in a number of built-in ways, such as:

    -  <code>**pandas.merge**</code>/<code>**pandas.join**</code> connects rows of two objects based on one or more keys. This is equivalent to **join** operations on relational databases.

    -  <code>**pandas.concat**</code> concatenates or "stacks" together objects along a specific axis, if any (there is only a single possible axis of concatenation for <code>**Series**</code>).

-  Full API documentation is available [here](https://pandas.pydata.org/pandas-docs/stable/merging.html)

## Database-style <code>DataFrame</code> Joins

In [58]:
"""
Merge or join operations combine data sets by linking rows using one or more keys. 
These operations are central to relational databases (e.g., SQL-based). 
The 'merge' function in pandas is the main entry point for achieving this.
"""
# Let's go back to our original DataFrame
del users['salary']
del users['age_interval']
print(users.head())

         age gender  occupation zip_code
user_id                                 
1         24      M  technician    85711
2         53      F       other    94043
3         23      M      writer    32067
4         24      M  technician    43537
5         33      F       other    15213


## What is a Join/Merge Operation?

-  Generally speaking, "joining" ("merging") two datasets is the process of bringing two datasets together into one, and aligning the rows from each based on common attributes or columns.

-  Typical operation in relational databases using <code>**SQL JOIN**</code> operator.

-  In <code>**pandas**</code> there are two different functions <code>**merge**</code> and <code>**join**</code>, both of which do similar things; the former is used for column-to-column joins, whereas the latter is more efficient when joining <code>**DataFrame**</code> objects on their indices.

## The Anatomy of a Join/Merge Operation

```python
merged_df = pd.merge(left_df, right_df, 
                     left_on=["l_col_1",...,"l_col_n"], 
                     right_on=["r_col_1",...,"r_col_n"],
                     how="{left|right|inner|outer}"
```

## The Anatomy of a Join/Merge Operation

-  <code>**left_df**</code> and <code>**right_df**</code> are the two <code>**DataFrame**</code> objects we want to merge.

-  <code>**left_on**</code> and <code>**right_on**</code> indicate the column(s) used for the merging operation on the left and right <code>**DataFrame**</code> objects, respectively (alternatively, use just <code>**on=[col_1,...,col_n]**</code> if column names are the same on both <code>**DataFrame**</code>s).

-  <code>**how**</code> is used to specify which kind of merge operation needs to be performed, i.e., one of: <code>**left**</code>, <code>**right**</code>, <code>**inner**</code> (default), and <code>**outer**</code>.

## Different Types of Join/Merge Operations

(./img/join_types.jpg)

<center>[Image Source](https://www.shanelynn.ie/merge-join-dataframes-python-pandas-index-1/)</center>

-  **Inner Merge/Inner Join** (default): Keep only those rows where the value which we want to merge on exists in **both** the **left** and **right** <code>**DataFrame**</code>s.

-  **Left Merge/Left (Outer) Join**: Keep every row in the **left** <code>**DataFrame**</code>. If some values of the column we are merging on are missing in the **right** <code>**DataFrame**</code>, add <code>**NaN**</code> to the result.

-  **Right Merge/Right (Outer) Join**: Keep every row in the **right** <code>**DataFrame**</code>. If some values of the column we are merging on are missing in the **left** <code>**DataFrame**</code>, add <code>**NaN**</code> to the result.

-  **Outer Merge/Full (Outer) Join**: Return **all** the rows from the **left** and the **right** <code>**DataFrame**</code>s, matches up rows where possible, otherwise add <code>**NaN**</code>.

In [59]:
# Suppose we have another DataFrame containing the (average) salary for some occupations. 
# Let's assume we don't have salary information for 'technician' and 'other'
# but we do for two extra occupations (e.g., 'gardener' and 'professor')
occupation_salary_data = {'occupation': ['administrator', 'writer', 'executive', 
                                       'student', 'lawyer', 'educator', 
                                       'scientist', 'entertainment', 
                                       'programmer', 'librarian', 'homemaker', 
                                       'artist', 'engineer', 'marketing', 
                                       'none', 'healthcare', 'retired', 
                                       'salesman', 'doctor', 'gardener', 
                                       'professor'],
        'salary': [150000, 40000, 300000, 1300, 27500, 45000, 
                      60000, 185000, 55000, 22000, 240000, 72000, 
                      91000, 66000, 0, 41000, 52000, 48000, 
                      140000, 16000, 82000]}

occupation_salary = pd.DataFrame(occupation_salary_data)
print("Occupation-Salary data:\n{}".format(occupation_salary.head()))

Occupation-Salary data:
      occupation  salary
0  administrator  150000
1         writer   40000
2      executive  300000
3        student    1300
4         lawyer   27500


In [60]:
users

,age,gender,occupation,zip_code
user_id,,,,
1,24,M,technician,85711
2,53,F,other,94043
3,23,M,writer,32067
4,24,M,technician,43537
5,33,F,other,15213
...,...,...,...,...
939,26,F,student,33319
940,32,M,administrator,02215
941,20,M,student,97229


In [61]:
"""
By default, 'merge' tries to join DataFrames on the basis of common column names.
In our example there is a column called 'occupation', which is common to both DataFrames.
Moreover, 'merge' implements an INNER JOIN, which means that the resulting DataFrame 
will contain all and only those records which match on both the left and right DataFrame. 
"""
merged = pd.merge(users, occupation_salary)
# The above is equivalent to:
# merged = users.merge(occupation_salary)
# Or:
# merged = pd.merge(users, occupation_salary, 
#                   left_on="occupation", right_on="occupation", how="inner")
# Or, again:
# merged = pd.merge(users, occupation_salary, on="occupation", how="inner")

In [62]:
merged

,age,gender,occupation,zip_code,salary
0,23,M,writer,32067,40000
1,26,M,writer,30068,40000
2,25,M,writer,40206,40000
3,32,M,writer,55369,40000
4,21,M,writer,52245,40000
...,...,...,...,...,...
806,29,M,doctor,63108,140000
807,51,M,doctor,85258,140000
808,45,M,doctor,47401,140000
809,64,M,doctor,97405,140000


In [63]:
print("Number of records in the left DataFrame: {}"
      .format(users.shape[0]))
print("Number of records in the right DataFrame: {}"
      .format(occupation_salary.shape[0]))
print("Unique values of 'occupation' in the left DataFrame:\n{}".
      format(users.occupation.unique()))
print("Unique values of 'occupation' in the right DataFrame:\n{}".
      format(occupation_salary.occupation.unique()))

Number of records in the left DataFrame: 943
Number of records in the right DataFrame: 21
Unique values of 'occupation' in the left DataFrame:
['technician' 'other' 'writer' 'executive' 'administrator' 'student'
 'lawyer' 'educator' 'scientist' 'entertainment' 'programmer' 'librarian'
 'homemaker' 'artist' 'engineer' 'marketing' 'none' 'healthcare' 'retired'
 'salesman' 'doctor']
Unique values of 'occupation' in the right DataFrame:
['administrator' 'writer' 'executive' 'student' 'lawyer' 'educator'
 'scientist' 'entertainment' 'programmer' 'librarian' 'homemaker' 'artist'
 'engineer' 'marketing' 'none' 'healthcare' 'retired' 'salesman' 'doctor'
 'gardener' 'professor']


In [64]:
print("Number of records in the resulting merged DataFrame: {}"
      .format(merged.shape[0]))
print(merged.head())

Number of records in the resulting merged DataFrame: 811
   age gender occupation zip_code  salary
0   23      M     writer    32067   40000
1   26      M     writer    30068   40000
2   25      M     writer    40206   40000
3   32      M     writer    55369   40000
4   21      M     writer    52245   40000


In [66]:
"""
Double check if the number of resulting rows after (inner) merge is compliant with
what we expect. Remember, inner merge results in a number of records which correspond
to the intersection of values on which we merge the 2 DataFrames on.
"""
# Let's see how many occupation values in the left DataFrame (users) 
# appear also in the right DataFrame (occupation_salary)
print("Number of 'occupation' values shared between the two DataFrames:\n{}"
      .format(users.occupation.isin(occupation_salary.occupation).value_counts()))

Number of 'occupation' values shared between the two DataFrames:
True     811
False    132
Name: occupation, dtype: int64


In [67]:
"""
Suppose we want to keep the records from the left DataFrame
"""
merged_left = pd.merge(users, occupation_salary, how='left')
print(merged_left.head())

   age gender  occupation zip_code   salary
0   24      M  technician    85711      NaN
1   53      F       other    94043      NaN
2   23      M      writer    32067  40000.0
3   24      M  technician    43537      NaN
4   33      F       other    15213      NaN


In [68]:
"""
How many records do we expect the resulting merged DataFrame above to have?
"""
print("Actual number of records: {}".format(merged_left.shape[0]))

Actual number of records: 943


In [69]:
"""
Suppose we want to keep the records from the right DataFrame
"""
merged_right = pd.merge(users, occupation_salary, how='right')
print(merged_right.head())

    age gender     occupation zip_code  salary
0  57.0      M  administrator    91344  150000
1  36.0      M  administrator    05201  150000
2  38.0      F  administrator    42141  150000
3  30.0      M  administrator    17870  150000
4  45.0      M  administrator    12550  150000


In [70]:
"""
How many records do we expect the resulting merged DataFrame above to have?
"""
print("Actual number of records: {}".format(merged_right.shape[0]))

Actual number of records: 813


In [71]:
"""
When we use 'merge' on column(s)-to-column(s) 
the indexes associated with the merging DataFrame objects are discarded.
"""
# Suppose we want to re-assign an Index object to the new merged DataFrame
# using the previous non-default index of the left DataFrame (users)
merged.index.name = 'user_id'

merged.index = users.index[users.occupation.isin(occupation_salary.occupation)]
print(merged.head())

         age gender occupation zip_code  salary
user_id                                        
3         23      M     writer    32067   40000
6         26      M     writer    30068   40000
7         25      M     writer    40206   40000
8         32      M     writer    55369   40000
9         21      M     writer    52245   40000


In [72]:
# Suppose we want to create another DataFrame containing the (average) salary
# for a set of 100 random users (i.e., not occupations)
# 1. Let's pick 100 user_ids uniformly at random
np.random.seed(23)

random_users = np.random.choice(users.index, 100, replace=False)

print("Selected number of random users = {}"
      .format(random_users.size))

print("Q: Is there any duplicates? A: {}"
      .format(pd.Series(random_users).duplicated().any()))

# print("Q: Is there any duplicates? A: {}"
#       .format())

Selected number of random users = 100
Q: Is there any duplicates? A: False


In [73]:
# 2. Let's do the same with the average salary.
# This time we extract a 100-sample drawn from a Normal distribution.
# To do so, we need to specify the two parameters of the distribution: 
# mean (mu) and standard deviation (sigma)
mu = 35000
sigma = 10000
normal_salaries = np.random.normal(mu, sigma, 100)

# Let's also round salaries to the 2nd decimal digit
normal_salaries = np.round(normal_salaries, decimals=2)
print(normal_salaries)

[38998.   31536.25 24355.11 30103.51 35611.84 17615.86 30250.3  42118.84
 45847.16 29421.92 39273.15 15920.3  36359.8  30626.69 48296.49 49052.06
 37141.21 14254.57 17026.05 22536.88 31362.75 33050.27 28228.48 28927.89
 23757.09 12374.72 36929.46 52598.52 33504.45 24091.63 34289.28 27012.23
 49070.25 48796.16 50157.58 24038.21 23184.02 29043.14 29916.06 35666.19
 38366.95 47605.43 40696.15 48960.02 25552.93 18293.37 46354.33 32787.86
 20396.14 48080.51 44002.58  3036.52 36445.56 20929.12 20650.53 31487.91
 28289.03 46646.2  35364.38 40477.7  27323.17 38161.09 30423.14 34731.28
 33913.6  38928.43 51416.31 49240.69 24231.9  35424.29 27401.46 42575.65
 47567.94 39560.88 46178.75 20353.03 43897.89 55995.4  57611.83 31803.22
 20234.6  35326.04 21825.77 46465.83 41483.11 40326.54 39110.58 42819.58
 30729.51 29656.11 29773.18 30663.94 28703.19 26390.94 58986.96 24663.34
 45841.04 32877.53 35263.56 29464.62]


In [74]:
# 2. Let's do the same with the average salary.
# This time we extract a 100-sample drawn from a Normal distribution.
# To do so, we need to specify the two parameters of the distribution: 
# mean (mu) and standard deviation (sigma)
mu = 35000
sigma = 10000
normal_salaries = np.random.normal(loc=mu, scale=sigma, size=100)
# Let's also round salaries to the 2nd decimal digit
normal_salaries = np.round(normal_salaries, decimals=2)
# Alternatively:
# np.round(normal_salaries, decimals=2, out=normal_salaries)

In [75]:
# 3. Let's now create the dictionary containing two keys: 'user_id' and 'salary'
# For each key, we associate the list of random_users and normal_salaries, respectively
user_to_salary_data = {'user_id': random_users, 'salary': normal_salaries}

# 4. We create the corresponding pandas DataFrame object
user_to_salary = pd.DataFrame(user_to_salary_data)

user_to_salary.set_index('user_id', inplace=True)

print("User-Salary data:\n{}".format(user_to_salary.head()))

User-Salary data:
           salary
user_id          
640      29778.94
320      35025.35
877      40907.00
772      41766.32
316      43400.22


In [76]:
# 5. Finally, merge (join) the original DataFrame (users) with this new one
# Merging uses the index this time, rather than the column as before
merged = pd.merge(users, user_to_salary, left_index=True, right_index=True)

# Note that if you don't specify that you are merging on indexes, 
# merge raises an error!
print(merged.head())

         age gender occupation zip_code    salary
user_id                                          
22        25      M     writer    40206  43972.37
25        39      M   engineer    55107  27868.12
30         7      M    student    55436  28235.88
54        22      M  executive    66315  39189.23
64        32      M   educator    43202  38699.79


In [77]:
# 5. Finally, merge (join) the original DataFrame (users) with this new one
# Merging uses the index this time, rather than the column as before
#merged = pd.merge(users, user_to_salary, left_index=True, right_index=True)
# Note that if you don't specify that you are merging on indexes, 
# merge raises an error!
# This is equivalent to (join is 'left' by default):

merged = users.join(user_to_salary, how='inner')

print(merged.head())

         age gender occupation zip_code    salary
user_id                                          
22        25      M     writer    40206  43972.37
25        39      M   engineer    55107  27868.12
30         7      M    student    55436  28235.88
54        22      M  executive    66315  39189.23
64        32      M   educator    43202  38699.79


## <code>pandas.merge</code> Arguments (1 of 2)

<p align="center">
  <img src="./img/pd_merge_args_1.png">
</p>

## <code>pandas.merge</code> Arguments (1 of 2)

<p align="center">
  <img src="./img/pd_merge_args_2.png">
</p>

## Options for the <code>how</code> Argument

<p align="center">
  <img src="./img/pd_merge_how.png">
</p>

## Concatenating <code>DataFrame</code>s

In [86]:
"""
We can use the 'concat' function in pandas to append either rows or columns (if any) 
from one object to another. 
Let's grab two subsets of our DataFrame to see how this works.
"""
# Read in first 5 rows of users table
users_first_5 = users.head()
print("First 5 records of 'users':\n{}".format(users_first_5))
print()
# Read in the last 5 rows
users_last_5 = users[-5:]
print("Last 5 records of 'users':\n{}".format(users_last_5))

First 5 records of 'users':
         age gender  occupation zip_code
user_id                                 
1         24      M  technician    85711
2         53      F       other    94043
3         23      M      writer    32067
4         24      M  technician    43537
5         33      F       other    15213

Last 5 records of 'users':
         age gender     occupation zip_code
user_id                                    
939       26      F        student    33319
940       32      M  administrator    02215
941       20      M        student    97229
942       48      F      librarian    78209
943       22      M        student    77841


In [87]:
"""
We can use the 'concat' function in pandas to append either rows or columns (if any) 
from one object to another. 
Let's grab two subsets of our DataFrame to see how this works.
"""
# Read in first 5 rows of users table
users_first_5 = users.head()
print("First 5 records of 'users':\n{}".format(users_first_5))
print()
# Read in the last 5 rows
users_last_5 = users[-5:]
print("Last 5 records of 'users':\n{}".format(users_last_5))

First 5 records of 'users':
         age gender  occupation zip_code
user_id                                 
1         24      M  technician    85711
2         53      F       other    94043
3         23      M      writer    32067
4         24      M  technician    43537
5         33      F       other    15213

Last 5 records of 'users':
         age gender     occupation zip_code
user_id                                    
939       26      F        student    33319
940       32      M  administrator    02215
941       20      M        student    97229
942       48      F      librarian    78209
943       22      M        student    77841


## Stack "_vertically_" vs. "_horizontally_": <code>axis</code>

-  By default, <code>**concat**</code> operates on <code>**axis=0**</code> (i.e., **rows**) and tells <code>**pandas**</code> to stack the second DataFrame under the first one (i.e., **vertically**). 
    -  In order for this to work, we need to make sure that both <code>**DataFrame**</code>s have the same column names and formats. 

-  Instead, <code>**axis=1**</code> will stack the **columns** in the second <code>**DataFrame**</code> to the right of the first one (i.e., **horizontally**). 
    - In this case, we want to make sure that the data we stack are related in some way.

In [88]:
"""
Stack DataFrames vertically (i.e., using default axis=0)
"""
# Stack the two DataFrames 'users_first_5' and 'users_last_5' on top of each other
vertical_stack = pd.concat([users_first_5, users_last_5], axis=0)
print(vertical_stack)

         age gender     occupation zip_code
user_id                                    
1         24      M     technician    85711
2         53      F          other    94043
3         23      M         writer    32067
4         24      M     technician    43537
5         33      F          other    15213
939       26      F        student    33319
940       32      M  administrator    02215
941       20      M        student    97229
942       48      F      librarian    78209
943       22      M        student    77841


In [89]:
"""
Stack DataFrames horizontally (i.e., using axis=1)
"""
# Place the two DataFrames 'users_first_5' and 'users_last_5' side by side
horizontal_stack = pd.concat([users_first_5, users_last_5], axis=1)
print(horizontal_stack)

          age gender  occupation zip_code   age gender     occupation zip_code
user_id                                                                       
1        24.0      M  technician    85711   NaN    NaN            NaN      NaN
2        53.0      F       other    94043   NaN    NaN            NaN      NaN
3        23.0      M      writer    32067   NaN    NaN            NaN      NaN
4        24.0      M  technician    43537   NaN    NaN            NaN      NaN
5        33.0      F       other    15213   NaN    NaN            NaN      NaN
939       NaN    NaN         NaN      NaN  26.0      F        student    33319
940       NaN    NaN         NaN      NaN  32.0      M  administrator    02215
941       NaN    NaN         NaN      NaN  20.0      M        student    97229
942       NaN    NaN         NaN      NaN  48.0      F      librarian    78209
943       NaN    NaN         NaN      NaN  22.0      M        student    77841


In [90]:
"""
Since the row indexes for the two DataFrames 'users_first_5' and 'users_last_5' 
are not the same, 'concat' can't place them next to each other (NaN values occur). 
A workaround for this is to reindex our second DataFrame using the 'reset_index()' method.
"""
# Set the index of the second DataFrame to that of the first one
users_last_5.set_index(users_first_5.index, inplace=True)
horizontal_stack = pd.concat([users_first_5, users_last_5], axis=1)
print(horizontal_stack)

         age gender  occupation zip_code  age gender     occupation zip_code
user_id                                                                     
1         24      M  technician    85711   26      F        student    33319
2         53      F       other    94043   32      M  administrator    02215
3         23      M      writer    32067   20      M        student    97229
4         24      M  technician    43537   48      F      librarian    78209
5         33      F       other    15213   22      M        student    77841


In [91]:
"""
Since the row indexes for the two DataFrames 'users_first_5' and 'users_last_5' 
are not the same, 'concat' can't place them next to each other (NaN values occur). 
A workaround for this is to reindex our second DataFrame using the 'reset_index()' method.
"""
# Set the index of the second DataFrame to that of the first one
users_last_5.set_index(users_first_5.index, inplace=True)
# Or:
# users_last_5 = users_last_5.set_index(users_first_5.index)
horizontal_stack = pd.concat([users_first_5, users_last_5], axis=1)
print(horizontal_stack)

         age gender  occupation zip_code  age gender     occupation zip_code
user_id                                                                     
1         24      M  technician    85711   26      F        student    33319
2         53      F       other    94043   32      M  administrator    02215
3         23      M      writer    32067   20      M        student    97229
4         24      M  technician    43537   48      F      librarian    78209
5         33      F       other    15213   22      M        student    77841


In [92]:
horizontal_stack['age']

,age,age
user_id,,
1,24,26
2,53,32
3,23,20
4,24,48
5,33,22


## <code>pandas.concat</code> Arguments

<p align="center">
  <img src="./img/pd_concat.png">
</p>
